# Building historical index constituents

This notebook demonstrates a workflow of building historical series of a given index constituents.

#### Learn more

To learn more about the Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
this [Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The LSEG Developer Community will be happy to help. 


### Imports

In [1]:
import refinitiv.data as rd
import pandas as pd
import copy

### Opening the session

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x12ebc0640 {name='workspace'}>

### Defining index and the date range

In [3]:
ric = '.FTSE'
start_date = '2014-01-01'
end_date = '2024-02-15'

### Getting the initial list of constituents as of the start date

In [4]:
def get_constutents_as_of(ric, date):
    const_df = rd.get_data(universe=[f"0#{ric}({date.replace('-', '')})"], 
                fields=["TR.PriceClose"],
                parameters={"SDATE":f"{date}", "EDATE":f"{date}"}
            )
    return const_df

In [5]:
initial_constituents_df = get_constutents_as_of(ric, start_date)
initial_constituents_df

,Instrument,Price Close
0,ABF.L,2414.527895
1,SAB.L^J16,3101.0
2,ABDN.L,366.412923
3,ADN.L^H17,500.0
4,ADML.L,1001.15354
...,...,...
96,WEIR.L,2132.0
97,WTB.L,3222.566622
98,WMH.L^D21,401.9
99,MRW.L^J21,242.175539


Below we extract the rics into a list:

In [6]:
initial_constituents = initial_constituents_df['Instrument'].to_list()
len(initial_constituents)

101

### Getting the Joiners and Leavers during the requested period

In [7]:
def get_constituent_changes(ric, start_date, end_date):

    const_changes  = rd.get_data(universe=[ric], 
                fields = ["TR.IndexJLConstituentRIC", "TR.IndexJLConstituentRIC.date",  "TR.IndexJLConstituentRIC.instrument",
                          "TR.IndexJLConstituentRIC.change","TR.IndexJLConstituentName"],
                parameters={"SDATE":f"{start_date}","EDATE":f"{end_date}", 'IC':'B'}
            )
    return const_changes

In [8]:
constitent_changes = get_constituent_changes(ric, start_date, end_date)
constitent_changes

,Instrument,Constituent RIC,Date,Instrument,Change,Constituent Name
0,.FTSE,SJP.L,2014-03-24,.FTSE,Joiner,St James's Place
1,.FTSE,BDEV.L,2014-03-24,.FTSE,Joiner,Barratt Dev
2,.FTSE,AMFW.L^J17,2014-03-24,.FTSE,Leaver,Amec Foster
3,.FTSE,TATE.L,2014-03-24,.FTSE,Leaver,Tate & Lyle
4,.FTSE,INTUP.L^G20,2014-06-23,.FTSE,Joiner,Intu Prop
...,...,...,...,...,...,...
210,.FTSE,HWDN.L,2023-09-25,.FTSE,Joiner,Howden Joinery
211,.FTSE,ICP.L,2023-12-18,.FTSE,Joiner,Intermediate Cap
212,.FTSE,HRGV.L,2023-12-18,.FTSE,Leaver,Hargreaves
213,.FTSE,DPH.L^A24,2024-01-16,.FTSE,Leaver,Dechra Pharms


### Defining helper functions to add joiners and remove leavers

In [9]:
def add_joiner(init_list, joiner_list):
    for joiner in joiner_list:
        if joiner not in init_list:
            init_list.append(joiner)
        else:
            print(f'{joiner} joiner is already in the list')
    return init_list

In [10]:
def remove_leaver(init_list, leaver_list):
    for leaver in leaver_list:
        if leaver in init_list:
            init_list.remove(leaver)
        else:
            print(f'{leaver} leaver is not in the list')
    return init_list

### Defining function to build historical time series of the index constituent

In [11]:
def get_historical_constituents(start_date, constituents, constitent_changes):
    
    hist_constituents = {start_date: constituents}
    for date in constitent_changes['Date'].unique():
        const_changes_date = constitent_changes[constitent_changes['Date'] == date]
        joiners = const_changes_date[const_changes_date['Change']=='Joiner']['Constituent RIC'].to_list()
        leavers = const_changes_date[const_changes_date['Change']=='Leaver']['Constituent RIC'].to_list()
        if len(leavers) > 0:
            constituents = remove_leaver(constituents, leavers)
        if len(joiners) > 0:
            constituents = add_joiner(constituents, joiners)
        new_constituents = copy.deepcopy(constituents)
        hist_constituents[str(date)[:10]] = new_constituents
    return hist_constituents

In [12]:
historical_constituents = get_historical_constituents(start_date, initial_constituents, constitent_changes)
historical_constituents

{'2014-01-01': ['ABF.L',
  'ADML.L',
  'AAL.L',
  'ANTO.L',
  'AHT.L',
  'AZN.L',
  'AV.L',
  'BAES.L',
  'BARC.L',
  'BP.L',
  'BATS.L',
  'BT.L',
  'BNZL.L',
  'BRBY.L',
  'CCH.L',
  'CPG.L',
  'DGE.L',
  'EXPN.L',
  'GLEN.L',
  'GSK.L',
  'HSBA.L',
  'ICAG.L',
  'IMB.L',
  'IHG.L',
  'ITRK.L',
  'LAND.L',
  'LGEN.L',
  'LLOY.L',
  'LSEG.L',
  'MNDI.L',
  'NG.L',
  'NWG.L',
  'NXT.L',
  'PSON.L',
  'PRU.L',
  'RKT.L',
  'REL.L',
  'RIO.L',
  'RR.L',
  'SGE.L',
  'SBRY.L',
  'SDR.L',
  'SVT.L',
  'SHEL.L',
  'SN.L',
  'SSE.L',
  'STAN.L',
  'TSCO.L',
  'ULVR.L',
  'UU.L',
  'VOD.L',
  'WTB.L',
  'WPP.L',
  'SJP.L',
  'III.L',
  'TW.L',
  'BDEV.L',
  'DCC.L',
  'FLTRF.L',
  'INF.L',
  'SMIN.L',
  'CRDA.L',
  'SKG.L',
  'RTO.L',
  'SMT.L',
  'SGRO.L',
  'BKGH.L',
  'SMDS.L',
  'HLMA.L',
  'MRON.L',
  'OCDO.L',
  'RMV.L',
  'SPX.L',
  'AUTOA.L',
  'PHNX.L',
  'JD.L',
  'MNG.L',
  'FRES.L',
  'KGF.L',
  'ENT.L',
  'BMEB.L',
  'PSHP.L',
  'RS1R.L',
  'AAF.L',
  'EDV.L',
  'CNA.L',
  'UTG.L

Above we have a dictionary of index constituents change dates with respective constituent list. Below we create a dataframe by merging the consitutents lists by adding the respective dates.

In [13]:
historical_constituents_df = pd.DataFrame()
for date, rics in historical_constituents.items():
    df = pd.DataFrame(rics, columns = ['rics'])
    df['date'] = date
    historical_constituents_df = pd.concat([historical_constituents_df, df])
historical_constituents_df.reset_index(inplace = True, drop=True)
historical_constituents_df

,rics,date
0,ABF.L,2014-01-01
1,ADML.L,2014-01-01
2,AAL.L,2014-01-01
3,ANTO.L,2014-01-01
4,AHT.L,2014-01-01
...,...,...
7559,HIK.L,2024-01-16
7560,MKS.L,2024-01-16
7561,HWDN.L,2024-01-16
7562,ICP.L,2024-01-16


### Closing the session

In [14]:
rd.close_session()